In [ ]:
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install jiwer
!pip install soundfile
!pip install librosa
!pip install evaluate
!pip install torchaudio

In [ ]:
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

In [ ]:
# from numba import cuda 
# device = cuda.get_current_device()
# device.reset()

In [1]:
from transformers import pipeline
from datasets import load_dataset, load_metric, Audio, ClassLabel, load_from_disk, Features, Value
import evaluate

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!git lfs

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log


In [4]:
print("----------------- Loading Datasets... -----------------")
features = Features(
    {
        "Unnamed: 0": Value("int32"),
        "Root": Value("string"),
        "SpeakerID": Value("string"),
        "Component": Value("string"),
        "Group": Value("int32"),
        "Age": Value("int32"),
        "Gender": Value("string"),
        "CEF": Value("string"),
        "DialectRegion": Value("string"),
        "Duration (seconds)": Value("float"),
        "Duration (days)": Value("float"),
        "Sentence": Value("string"),
        "Path": Audio(sampling_rate=48_000),
    }
)

# To prepare the csv:
# add column 'audio' with absolut path to the audio files (Excel function: =CONCAT('path/to/audio_files/folder' B2))
test_from_csv = load_dataset('csv', data_files={'test':"/path/file.csv", },)
                                   # data_dir="/path/")
# trainset = DatasetDict()
# file = open("/path/file.json")
# spl = json.load(file)
# train = load_dataset('csv', data_files={'test': "/path/file.csv"})

# trainset['train'] = train["test"]

# dataset = DatasetDict()
# test_set = trainset["train"].select(spl['1']['7'])
test_from_csv = test_from_csv.cast(features)

test_from_csv = test_from_csv.remove_columns(
    ["Unnamed: 0","Root", "DialectRegion", "Age", "SpeakerID", "Group", "Gender", "Duration (seconds)", "Duration (days)", "CEF", "Component"])

test = test_from_csv['test']

test['Path']
print("----------------- Loading Datasets complete. -----------------\n\n")

----------------- Loading Datasets... -----------------
----------------- Loading Datasets complete. -----------------




In [5]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["Sentence"]).lower()
    return batch

test = test.map(remove_special_characters)

In [6]:
from transformers import pipeline
from datasets import load_dataset, load_metric, Audio, ClassLabel, load_from_disk, Features, Value
import evaluate
import pandas as pd

transcriber = pipeline("automatic-speech-recognition", model='modddddel/whisper-native-children-5-dutch', device=2,
                      generate_kwargs = {"language":"<|nl|>","task": "transcribe"},
                      )

wer = evaluate.load("wer")
labels = []
preds = []
i = 0

for recording in test:
    label = labels.append(recording['Sentence'])
    pred = preds.append(transcriber(recording['Path'])['text'])
    i += 1
    print(str(i) + '/' + str(len(test)))
    print('predicted: ' + preds[i-1])
    print('actual: ' + labels[i-1])

# Recommended: save the results in a CSV file to use later for comparison
# (to avoid having to run the model(s) again)
df = pd.DataFrame({'reference': labels, 'hypothesis': preds})
df.to_csv('/path/file.csv')

print( 100 * wer.compute(predictions=preds, references=labels))

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/112k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

1/1090
predicted: mmm naar marrakish het is een uh druk stad het is een toeristische stad het altijd moet met mooi weer en je hebt winkelers die 24 uur openblijven ja en uh wat is het een beetje leuke sfeer mmm hoe moet k t gesmeen hoe moet k t zeggen het is een mooie stad het is een druk stad toeristische stad met uh winkelers die 24 uur openblijven en t wordt veel bezocht en uh je hebt mooie uh
actual: uhm naar Marrakech. 't is een uh drukke stad. het is een toeristische stad. het altijd mo met mooi weer en je hebt winkels die vierentwintig uur uh open blijven. ja en uh dat is het een beetje. leuke sfeer. mmm. . het is een mooi stad. het is een drukke stad toeristische stad. met uh winkels die vierentwintig uur uh open blijven. en 't wordt veel bezocht en uh mooie
2/1090
predicted: zelf bekijken het boekje na jaar of voor jaar niet zo niet zo hot ja ik vind uh de hoogste te hoge temperatuur vind ik niet zo leuk mmm misschien de fotocamera ja zeker ja zeker mijn man mijn moeder en mij

/gpfs/home2/gshekoufand/pvenv/lib/python3.9/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


11/1090
predicted: michel anselo uh monument is is vatikan ook een monument  een vatikan uh die andere uh monument heb ik geen uhm ik ve ik vergeet de naam  uh er zijn zoveel  monumenten in roma maar ik uh ik vergeet de de namen van die monument één is die uh michel anselo de andere is ook de vatikan andere
actual: Michaelangelo. uh... monument.  is is Vaticaan ook een uh monument?  een uh Vaticaan. uh die andere uh monument heb ik geen uhm ik ik vergeet de naam. nja. uh er zijn zoveel  monumenten in Rome maar ik uh ik vergeet de de namen van die monument. één is die uh Michaelangelo. de andere is ook de Vaticaan. andere...
12/1090
predicted: heel mooi museums nu moe z mmm museum  ik weet ik weet jij kan hobby lezen uh spelletjes met mij vriend ik weet ik weet het niet precies maar twee of uh drie keer per weken lezen deze week oh oh uh nee ay wat is het geweest uhm ik weet niet ik weet ik weet het niet uhm wijn
actual: heel mooi moesiems mur moes museum. ach. ik weet . ja ik kan. hobb

In [7]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = '/path/file.csv'
df = pd.read_csv(file_path)

# Apply the regex operation to the 'hypothesis' column
df['hypothesis'] = df['hypothesis'].replace(r'�\s*', '', regex=True)

# Save the modified DataFrame to a new CSV file
output_file_path = "/path/file.csv"
df.to_csv(output_file_path, index=False)


In [8]:
import jiwer
import pandas as pd

df = pd.read_csv('/path/file.csv')

out = jiwer.process_words(
    df['reference'].values.tolist(),
    df['hypothesis'].values.tolist(),
)

print(jiwer.visualize_alignment(out))

sentence 1
REF: uhm naar Marrakech.  't is een uh drukke stad. het is een toeristische stad. het altijd   mo met mooi weer en je hebt   winkels die vierentwintig uur          uh open blijven. ja en uh dat is het een beetje. leuke ***** *** *** **** * * ******* *** **** sfeer. mmm.      . het is een  mooi stad. het is een drukke stad toeristische stad. met uh   winkels die vierentwintig uur          uh open blijven. en 't wordt veel bezocht en uh ** **** mooie **
HYP: mmm naar  marrakish het is een uh   druk  stad het is een toeristische  stad het altijd moet met mooi weer en je hebt winkelers die            24 uur openblijven **** ******** ja en uh wat is het een  beetje leuke sfeer mmm hoe moet k t gesmeen hoe moet      k    t zeggen het is een mooie  stad het is een   druk stad toeristische  stad met uh winkelers die            24 uur openblijven **** ******** en  t wordt veel bezocht en uh je hebt mooie uh
       S               S   S                S     S                          

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = '/gpfs/home2/gshekoufand/Data/data/meta/csv/nl/training/nc/results/results9_a.csv'
df = pd.read_csv(file_path)

# Apply the regex operation to the 'hypothesis' column
df['hypothesis'] = df['hypothesis'].replace(r'�\s*', '', regex=True)

# Save the modified DataFrame to a new CSV file
output_file_path = "/gpfs/home2/gshekoufand/Data/data/meta/csv/nl/training/nc/results/results9_a_modified.csv"
df.to_csv(output_file_path, index=False)


In [ ]:
import jiwer
import pandas as pd

df = pd.read_csv('/gpfs/home2/gshekoufand/Data/data/meta/csv/nl/training/nc/results/results9_a_modified.csv')

out = jiwer.process_words(
    df['reference'].values.tolist(),
    df['hypothesis'].values.tolist(),
)

print(jiwer.visualize_alignment(out))